# STA 141B Data & Web Technologies for Data Analysis

### Lecture 17, 11/27/24, SQL

### Announcements

 - No lecture on Thursday!

### Last week's topics

- NLP

### Today's topics
- Databases
- Structured Query Language
 
### Datasets

* The Suppliers Database

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](https://www.sqltutorial.org/sql-cheat-sheet/)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

## Databases

A _database_ is a collection of data. There are several different models for how to organize data in a database; these are called _database models_. In this context, "model" refers to a design or mental model, not a statistical model.

The _relational model_ organizes data as a collection of tables. Tables have rows (also called _tuples_ or _records_) and columns (also called _attributes_). Most tables have a _key_ column that is unique for each row and _relates_ the table to other tables. The relational model is the most popular database model by far, and the one we'll focus on in this course.

There are also many different software programs for managing databases, called _database management systems_ (DBMS). Each DBMS usually has its own format for storing data on disk, independent of the database model. Some popular DBMSes are:

* [SQLite](https://www.sqlite.org/)
* [MySQL](https://www.mysql.com/)
* [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server)
* [PostgreSQL](https://www.postgresql.org/)

Why use a database? There are several reasons:

* Your data may already be in a database, so converting to another format is extra work.
* Database operations are highly optimized, so they typically take less time and memory than an equivalent operation in Python.
* Database operations can run on datasets that are too large to fit in memory. Doing this in Python requires special programming strategies.
* Many DBMSes provide built-in version control, multi-user access, and security checks.
* Databases can be updated in real time.

## Structured Query Language

_Structured query language_ (SQL) is a language designed for querying information in relational databases.

A free SQL tutorial is available [here](https://www.w3schools.com/sql/).

### Getting Connected

There are several ways to connect to a database and run SQL queries from Python:

* The built-in __sqlite3__ module, which only supports SQLite.
* The __sqlalchemy__ package, a unified interface for a variety of different SQL database formats (more than just SQLite). See the [tutorial](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) for more details.

We'll use a SQLite database here, since SQLite is possibly [the most-used database engine in the world](https://sqlite.org/mostdeployed.html). SQLite's popularity is partly due to its reliability, easy setup, and broad range of features.

Let's connect to the suppliers database:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sqlite3 as sql

To connect to a database, use the module's `connect()` function. This is similar to opening a file; you should close the database when you're done using it.

In [ ]:
db = sql.connect("../data/suppliers.sqlite")

In [ ]:
db

To execute a SQL query, use the connection's `.execute()` method. This returns a _cursor_, which is a pointer to the results in the database (imagine a finger pointing at the results).

SQLite databases store metadata in a special table called `sqlite_master`. We can use `sqlite_master` to find out the names of the other tables in the database.

In [ ]:
cur = db.execute("SELECT * FROM sqlite_master")

In [ ]:
type(cur)

To get the results from the database, use one of the cursor's fetch methods. The `.fetchall()` method returns all rows in the result.

In [ ]:
cur.fetchall() 

By default, `sqlite3` will return rows as tuples. If you'd rather have the rows as dictionaries indexed by column name, set the `.row_factory` attribute on the database connection.

In [ ]:
db.row_factory = sql.Row 

Now the rows will behave like dictionaries:

In [ ]:
cur = db.execute("SELECT * FROM sqlite_master")
rows = cur.fetchall()
dict(rows[3])

In [ ]:
rows

Don't forget to close the database when you're done!

In [ ]:
#db.close()

We'll generally use the `pd.read_sql()` function in __pandas__ to run our SQL queries. 

The function takes a SQL query and an open database connection as arguments, so you still need to connect to the database first with `sqlite3` or `sqlalchemy`. The result of the query is returned as a data frame.

### `SELECT`

The `SELECT` command selects rows from a table. Most of your SQL queries will start with `SELECT`. The syntax is:

```sql
SELECT col1, col2, ... FROM my_table;
```

Here `col1`, `col2`, and so on are column names and `my_table` is a table name. You can select all columns with an asterisk  `*`.

SQL is not case-sensitive and ignores whitespace, but the convention is to write SQL keywords in uppercase and column/table names in lowercase. A semicolon `;` marks the end of a SQL query, but this is optional for many tools.

In [ ]:
# db.excute("SELECT * FROM file_name")

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db) # ; is optional but is preferred

In [ ]:
# select only name and type from the data
pd.read_sql("SELECT name, type FROM sqlite_master", db)

In [ ]:
pd.read_sql("SELECT * FROM suppliers;", db)

In [ ]:
pd.read_sql("SELECT * FROM parts;", db)

In [ ]:
pd.read_sql("SELECT * FROM supplierparts;", db)

### `LIMIT`

The `SELECT` command can be extended with many other keywords.

The first of these is `LIMIT`, which limits the number of rows returned. `LIMIT` is the SQL equivalent of Pandas' `.head()` method.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts LIMIT 3", db)

### `DISTINCT`

The `DISTINCT` keyword limits rows to distinct results. `DISTINCT` is the SQL equivalent of Pandas' `.drop_duplicates()` method.

Keep in mind that `DISTINCT` applies to all of the selected columns, not just one column.

In [ ]:
pd.read_sql("SELECT color, city FROM parts", db)

In [ ]:
pd.read_sql("SELECT DISTINCT color, city FROM parts;", db)

### `ORDER BY`

The `ORDER BY` keyword sorts the returned rows. `ORDER BY` is the SQL equivalent of Pandas' `.sort_values()` method.

In [ ]:
pd.read_sql("SELECT * FROM parts ORDER BY weight, color LIMIT 2", db)

Add the suffix `ASC` for an ascending sort (smallest to largest) and `DESC` for a descending sort (largest to smallest).

In SQLite, the default is ascending, but other other databases may differ.

In [ ]:
pd.read_sql("SELECT * FROM parts ORDER BY weight DESC, color ASC;", db)

In [ ]:
pd.read_sql("SELECT partid FROM parts ORDER BY weight DESC, city;", db)

### `WHERE`

`WHERE` puts conditions on the rows returned. `WHERE` is the SQL equivalent of subsetting.

You can use `=` to test equality. Other comparison operators, such as `>=`, are also available.

In [ ]:
pd.read_sql("SELECT * FROM parts", db)

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE weight >= 17 OR color = 'Green ';", db)

You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE (city = 'London' OR city = 'Paris') AND (color = 'Red' OR color = 'Green');", db)

You can use `IN` to check whether a value is in a collection of values.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE weight >= 12 AND weight <= 17;", db)

In [ ]:
pd.read_sql("SELECT * FROM parts;", db)

SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex (re) `.*`
* `_` matches any one character, similar to regex `.`

In other databases (but not SQLite):
* `[]` matches any one of the characters you put inside the brackets, identical to regex `[]`

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE city LIKE '_o%';", db)

The `BETWEEN` ... `AND` keyword is useful for selecting ranges.

In [ ]:
pd.read_sql("SELECT * FROM parts WHERE weight BETWEEN 12 AND 17 ORDER BY weight DESC, city;", db)

### Operators

You can use arithmetic operators `+`, `-`, `*`, `/`, `%` on SQL columns to perform columnwise computations. These are the SQL equivalent of vectorized arithmetic.

In [ ]:
pd.read_sql("SELECT weight+weight,* FROM parts", db) # the first *: multiplication, and the second *: reads all columns.

In [ ]:
pd.read_sql("SELECT weight + weight AS double_weight, * FROM parts", db)

In [ ]:
pd.read_sql('''
    SELECT weight + weight, * 
    FROM parts 
    WHERE weight * weight > 200;
''', db)

### `AS`

You can rename a column with the `AS` keyword. This keyword is especially useful together with SQL arithmetic operators and functions.

In [ ]:
pd.read_sql('''
    SELECT weight + weight AS weightsum, * 
    FROM parts 
    WHERE weight * weight > 200;
''', db)

### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [ ]:
pd.read_sql('''
    SELECT weight * 12 AS multiplied_weight, * 
    FROM parts;
''', db)

In [ ]:
pd.read_sql("SELECT SUM(weight * 12) AS avg_multiplied_weight FROM parts;", db)

In [ ]:
pd.read_sql("SELECT COUNT(*) AS numberrows FROM parts;", db) 

In [ ]:
pd.read_sql("SELECT LOWER(city) AS uppercity, * FROM parts;", db)

### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [ ]:
pd.read_sql("SELECT * FROM parts ORDER BY partname", db)

In [ ]:
pd.read_sql('''SELECT AVG(weight) AS avg_weight_per_city, partname, city FROM (
               SELECT * FROM parts ORDER BY partname
           ) GROUP BY city;''', db)

In [ ]:
pd.read_sql("SELECT AVG(weight), city, partname FROM parts GROUP BY city, partname;", db)

### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [ ]:
pd.read_sql('''
    SELECT *
    FROM (SELECT AVG(weight) AS avg_weight, city  FROM parts WHERE weight > 15 GROUP BY city)
    WHERE avg_weight > 17
''', db)

In [ ]:
pd.read_sql(''' 
    SELECT AVG(weight) AS avg_weight, city
    FROM parts WHERE weight > 15 GROUP BY city
    HAVING avg_weight > 17
''', db)

In [ ]:
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city HAVING avg_weight > 15", db)

### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts", db)

In [ ]:
pd.read_sql("""
    SELECT * 
    FROM supplierparts
    LEFT JOIN parts ON supplierparts.partid = parts.partid
""", db)

SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

In [ ]:
pd.read_sql("SELECT * FROM supplierparts", db)

In [ ]:
pd.read_sql("""
    SELECT * 
    FROM supplierparts AS l
    LEFT JOIN suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

In [ ]:
pd.read_sql("""
    SELECT l.qty, r.* 
    FROM supplierparts AS l
    LEFT JOIN suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

### Subqueries

You can write one or more `SELECT` queries within another:

In [ ]:
pd.read_sql("""
    SELECT * 
    FROM (
        SELECT l.*, r.supplierid, r.qty FROM
        parts AS l
        INNER JOIN
        supplierparts AS r
        ON l.partid = r.partid) AS a
    INNER JOIN
        (SELECT * FROM suppliers
        WHERE status > 10) AS b
    ON a.supplierid = b.supplierid;
""", db) 

You can paste strings into SQL queries, but make sure the string you paste in isn't compromised (e.g., user input)

![Drag Racing](https://imgs.xkcd.com/comics/exploits_of_a_mom_2x.png)

In [ ]:
val = pd.read_sql("SELECT AVG(weight) FROM parts", db).iloc[0, 0]
val

In [ ]:
query = "SELECT * FROM parts WHERE weight > " + format(val) #+ "; NEW QUERY"
query

In [ ]:
pd.read_sql(query, db)

### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

In [ ]:
result = db.execute("""
    CREATE TABLE table2 AS
        SELECT l.*, r.qty, r.supplierid FROM
            parts AS l
        INNER JOIN
            supplierparts AS r
        ON l.partid = r.partid
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM table2", db)

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db)

In [ ]:
db.execute("DROP TABLE table2;")
db.commit()

In [ ]:
db.close()

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///../data/suppliers.sqlite")
conn = engine.raw_connection()

In [ ]:
conn

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

In [ ]:
engine.table_names()

In [ ]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.